<a href="https://colab.research.google.com/github/Ilvecoding0912/DETR_Robotic_Surgery/blob/main/DETR_training_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DETR training process


Clone DETR Repo after changing to our dataset

In [1]:
!git clone https://github.com/Ilvecoding0912/DETR_Robotic_Surgery.git
%cd DETR_Robotic_Surgery

Cloning into 'DETR_Robotic_Surgery'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 118 (delta 10), reused 0 (delta 0), pack-reused 80
Receiving objects: 100% (118/118), 2.46 MiB | 18.37 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/DETR_Robotic_Surgery


Download DETR weights

In [2]:
import gdown

url = 'https://drive.google.com/uc?id=1HV2Tit0CsVeYKHugjx8QxROPegQa3AV-'
gdown.download(url,'detr_weights.pth',quiet=True)

'detr_weights.pth'

In [5]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
!pip -q install 'git+https://github.com/facebookresearch/segment-anything.git'

--2023-08-28 04:50:00--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.59, 13.227.219.70, 13.227.219.33, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  57.8MB/s    in 5.7s    

2023-08-28 04:50:06 (62.7 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]

  Preparing metadata (setup.py) ... done


In [5]:
%cd /content/DETR_Robotic_Surgery
from collections import OrderedDict
import torch
from torch import nn
from typing import Dict, List
from util.misc import NestedTensor
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch.nn.functional as F
from models.position_encoding import build_position_encoding

class backboneSAM(nn.Module):

    def __init__(self, ):
        super().__init__()
        sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
        #Freeze()
        sam.eval()
        for param in sam.image_encoder.parameters():
            param.requires_grad = False
        self.sam_encoder = sam.image_encoder


    def forward(self, tensor_list: NestedTensor):
        xs = OrderedDict()
        with torch.no_grad():
            xs['0'] = self.sam_encoder(tensor_list.tensors)
        # print('out sam encoder', xs['0'].shape)
        out: Dict[str, NestedTensor] = {}
        for name, x in xs.items():
            m = tensor_list.mask
            # print('m', m.shape)
            assert m is not None
            import torch.nn.functional as F
            mask = F.interpolate(m[None].float(), size=x.shape[-2:]).to(torch.bool)[0]
            # mask = F.upsample_nearest(m[None].float(),size=x.shape[-2:]).to(torch.bool)[0]
            # print('mobarak',name, x.shape, mask.shape)
            out[name] = NestedTensor(x, mask)
        return out

class Joiner(nn.Sequential):
    def __init__(self, backbone, position_embedding):
        super().__init__(backbone, position_embedding)

    def forward(self, tensor_list: NestedTensor):
        xs = self[0](tensor_list)
        out: List[NestedTensor] = []
        pos = []
        for name, x in xs.items():
            out.append(x)
            # position encoding
            pos.append(self[1](x).to(x.tensors.dtype))

        return out, pos

def build_backbone(args):
    position_embedding = build_position_encoding(args)
    train_backbone = args.lr_backbone > 0
    return_interm_layers = args.masks
    backbone = backboneSAM()#Backbone(args.backbone, train_backbone, return_interm_layers, args.dilation)
    model = Joiner(backbone, position_embedding)
    # model.num_channels = backbone.num_channels
    return model

/content/DETR_Robotic_Surgery


Part of content related to training in 'main.py'.
(We ignore the evaluation part.)

In [ ]:
%cd /content/DETR_Robotic_Surgery
from main import get_args_parser
import argparse
import torch
import time
import random
import datetime
import json
from torch.utils.data import DataLoader, DistributedSampler
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from pathlib import Path
import numpy as np
from engine import evaluate, train_one_epoch
from models import build_model
from datasets.coco import *
import os

def main():

    parser = argparse.ArgumentParser('DETR training and evaluation script', parents=[get_args_parser()])
    args = parser.parse_args([])
    args.output_dir = './outputs' # Results dir
    args.endovis_path = 'endovis17' # Path to our dataset
    args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # training parameters
    args.start_epoch = 0
    args.epochs = 50 # total number of epoch
    args.batch_size = 1

    # create output directoty if not exist
    if os.path.exists(args.output_dir) is False:
        os.mkdir(args.output_dir)

    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))
    print(args)

    device = torch.device(args.device)

    # fix the seed for reproducibility
    seed = args.seed + utils.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # create model
    model, criterion, postprocessors = build_model(args)
    model.to(device)
    # print(model)
    model_without_ddp = model
    n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('number of params:', n_parameters)

    param_dicts = [
        {"params": [p for n, p in model_without_ddp.named_parameters() if "backbone" not in n and p.requires_grad]},
        {
            "params": [p for n, p in model_without_ddp.named_parameters() if "backbone" in n and p.requires_grad],
            "lr": args.lr_backbone,
        },
    ]
    optimizer = torch.optim.AdamW(param_dicts, lr=args.lr,
                                    weight_decay=args.weight_decay)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

    # Our dataset class (initialize in datasets->coco.py)
    dataset_train = EnvidosDataset(args.endovis_path, transforms=make_coco_transforms('train'), mode='train', length=4)

    if args.distributed:
        sampler_train = DistributedSampler(dataset_train)
    else:
        sampler_train = torch.utils.data.RandomSampler(dataset_train)

    batch_sampler_train = torch.utils.data.BatchSampler(
        sampler_train, args.batch_size, drop_last=True)

    data_loader_train = DataLoader(dataset_train, batch_sampler=batch_sampler_train,
                                    collate_fn=utils.collate_fn, num_workers=args.num_workers)

    #---------------------- Training Process ----------------------
    print("Start training")
    start_time = time.time()

    sam_backbone = build_backbone(args)#BackboneBase()
    sam_backbone.to(device)
    sam_backbone.eval()
    model_without_ddp.backbone = sam_backbone
    model_without_ddp.input_proj = nn.Conv2d(256, model_without_ddp.transformer.d_model, kernel_size=1)
    model_without_ddp.to(device)

    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            sampler_train.set_epoch(epoch)

        #----------------- main training function (can be seen in 'engine.py') -----------------
        train_stats = train_one_epoch(
            model_without_ddp, criterion, data_loader_train, optimizer, device, epoch,
            args.clip_max_norm)

        lr_scheduler.step()

        # Save trained models
        output_dir = Path(args.output_dir)
        if args.output_dir:
            checkpoint_paths = [output_dir / f'checkpoint{epoch:04}.pth']
            if (epoch + 1) % args.lr_drop == 0 or (epoch + 1) % 5 == 0:
                checkpoint_paths.append(output_dir / f'checkpoint{epoch:04}_lr{args.lr}.pth')
            for checkpoint_path in checkpoint_paths:
                utils.save_on_master({
                    'model': model_without_ddp.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'lr_scheduler': lr_scheduler.state_dict(),
                    'epoch': epoch,
                    'args': args,
                }, checkpoint_path)

        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                        'epoch': epoch,
                        'n_parameters': n_parameters}

        if args.output_dir and utils.is_main_process():
            with (output_dir / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")


    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))


main()

/content/DETR_Robotic_Surgery
Not using distributed mode
git:
  sha: 620f6db10fc39148252ca1b758d02b916d9cdce1, status: has uncommited changes, branch: main

Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=1, weight_decay=0.0001, epochs=50, lr_drop=200, clip_max_norm=0.1, frozen_weights=None, model=True, backbone='resnet50', dilation=False, position_embedding='sine', enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, pre_norm=False, masks=False, aux_loss=True, set_cost_class=1, set_cost_bbox=5, set_cost_giou=2, mask_loss_coef=1, dice_loss_coef=1, bbox_loss_coef=5, giou_loss_coef=2, eos_coef=0.01, dataset_file='endovis17', coco_path=None, endovis_path='endovis17', coco_panoptic_path=None, remove_difficult=False, output_dir='./outputs', device='cuda', seed=42, resume='', start_epoch=0, eval=False, num_workers=2, world_size=1, dist_url='env://', distributed=False)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


number of params: 41281037
Start training
Epoch: [0]  [0/4]  eta: 0:00:04  lr: 0.000100  class_error: 66.67  loss: 31.3212 (31.3212)  loss_ce: 2.2468 (2.2468)  loss_bbox: 1.5440 (1.5440)  loss_giou: 0.9390 (0.9390)  loss_ce_0: 2.4712 (2.4712)  loss_bbox_0: 1.8375 (1.8375)  loss_giou_0: 1.2668 (1.2668)  loss_ce_1: 2.3951 (2.3951)  loss_bbox_1: 1.7656 (1.7656)  loss_giou_1: 1.2705 (1.2705)  loss_ce_2: 2.1937 (2.1937)  loss_bbox_2: 2.3438 (2.3438)  loss_giou_2: 1.4862 (1.4862)  loss_ce_3: 2.1639 (2.1639)  loss_bbox_3: 1.3836 (1.3836)  loss_giou_3: 1.2681 (1.2681)  loss_ce_4: 2.1586 (2.1586)  loss_bbox_4: 1.6999 (1.6999)  loss_giou_4: 0.8869 (0.8869)  loss_ce_unscaled: 2.2468 (2.2468)  class_error_unscaled: 66.6667 (66.6667)  loss_bbox_unscaled: 0.3088 (0.3088)  loss_giou_unscaled: 0.4695 (0.4695)  cardinality_error_unscaled: 97.0000 (97.0000)  loss_ce_0_unscaled: 2.4712 (2.4712)  loss_bbox_0_unscaled: 0.3675 (0.3675)  loss_giou_0_unscaled: 0.6334 (0.6334)  cardinality_error_0_unscaled: 97

In [ ]:
import torchvision.transforms as T
import numpy as np

invTrans = T.Compose([ T.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                T.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

img1 = invTrans(samples.tensors)
print(img1.shape)
print(img1[0].permute(1,2,0).shape)
img = img1[0].permute(1,2,0).cpu()
# ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

print(img.max(), img.min())
img_cv = np.array(img*255).astype(np.uint8)
im_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
cv2.imwrite('sample.png', im_rgb)